# LLM 원리 + OpenAI Chat Completion API 활용
---
### 학습 목표
1. LLM 기본 개념: 트랜스포머 아키텍처와 토큰 예측 원리 이해
2. OpenAI API 기초: Chat Completion API의 기본 사용법 습득
3. 멀티모달 활용: 텍스트, 이미지, 오디오를 처리하는 방법 학습
4. 구조화된 출력: JSON Schema를 사용한 구조화된 응답 생성
5. 매개변수 최적화: Temperature, Top-p등 주요 매개변수 활용
6. 다양한 API: Embeddings, Moderation, Function Calling, Streaming 활용
---
### 1. LLM 기본 개념
#### 1.1 LLM(Large Language Model)의 생성 원리
##### LLM은 어떻게 작동하나요?
- 트랜스포머 구조: 대화형 AI의 핵심 아키텍처
- 토큰 예측: 다음에 올 가장 적절한 단어를 예측
- 학습 방식: 인터넷의 방대한 텍스트 데이터로 사전 훈련

##### 핵심 프로세스
1. 토큰화: 텍스트를 작은 단위(토큰)로 분할
3. 확률 계산: 각 토큰이 다음에 올 확률 계산
3. 토큰 생성: 확률 분포에 따라 토큰 선택
4. 반복: 종료 조건까지 과정 반복

##### 트랜스포머란?
- 인코더-디코더 구조: 입력과 출력을 동시에 처리
- 어텐션 메커니즘: 입력의 모든 부분을 동시에 고려하여 중요한 정보에 집중
---
### 2. OpenAI API 핵심 개념
#### 1.1 주요 구성 요소
##### 1. 메시지 형식
```
messages = [
	{"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트 입니다."},
	{"role": "user", "content": "파이썬에서 리스트를 정렬하는 방법을 알려주세요."},
	{"role": "assistant", "content": "sort() 매서드나 sorted() 함수를 사용할 수 있습니다."},
]
```

#### 2. 현재 사용 가능한 주요 모델 (2025년 기준)
- gpt-4.1: 최고 성능, 복잡한 작업용
- gpt-4.1-mini: 빠른 속도, 비용 효율적
- gpt-4.1-nano: 초고속, 최저 비용
- o3, o4-mini: 복잡한 추론 작업용
- gpt-4o: 멀티모달(텍스트, 이미지, 오디오)
> 참고 : 모델 목록은 지속적으로 업데이트 됩니다. 최신 모델 정보는 [\[OpenAI Platform - Models\]](https://platform.openai.com/docs/models)에서 확인하세요.

##### 3. API 응답 구조
```
{
	"id": "chatcmpl-...",
	"object": "chat.completion",
	"model": "gpt-4.1-mini",
	"choices": [
		{
			"message": {
				"role": "assistant",
				"content": "생성된 텍스트",
			}
		}
	],
	"usage": {
		"prompt_tokens": 10,
		"completion_tokens": 50,
		"total_tokes": 60,
	}
}
```
---
### 3. 환경 설정
- uv 프로젝트 설정
	- 프로젝트 생성: `uv init [프로젝트명]`
	- 가상환경 생성: `uv venv --python=3.12`
	- 가상환경 활성화 : `.venv/bin/activate` (Unix) 또는 `.venv\Scripts\activate` (Windows) # 굳이 하지 않아도 된다.
- 패키지 설치
```
	# uv 사용 (권장)
    uv add langchain langchain_openai python-dotenv ipykernel

    # pip 사용
    pip install langchain langchain_openai python-dotenv ipykernel
```
- API 키 설정
```python
    # .env 파일 생성 - 환경변수는 무조건 대문자로 작성해야한다.
    OPENAI_API_KEY=your_api_key_here
```

```python
    # Python에서 로드
    from dotenv import load_dotenv
    import os

    load_dotenv()
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
```

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

---
### 4. 기본 사용법
#### 4.1 텍스트 답변을 생성
- OpenAI 클라이언트 설정
	- `from openai import OpenAI`로 OpenAI 패키지를 임포트
	- `client = OpenAI()`로 API 클라이언트 생성
	- API 키는 환경변수나 직접 설정을 통해 제공 가능
	- 보안을 위해 API키는 `.env` 파일이나 환경변수를 통해 관리하는 것을 권장
- Chat Completion 요청 구조
	- `client.chat.completions.create()`를 통해 텍스트 생성을 요청
	- `model`: 사용할 모델을 지정(예: "gpt-4.1-mini")
	- `messages`: 대화 맥락을 리스트 형태로 전달
		- `role`: "system", "user"등의 역할 지정
		- `content`: 실제 메시지 내용
	- `temperature`: 생성 텍스트의 무작위성 조절(0~1)
	- `max_tokens`: 생성될 최대 토큰 수 제한
- 응답 처리
	- API는 JSON형태로 응답을 반환한다.
	- `response.choices[0].message.content`: 생성된 분석 텍스트
	- `respnse.usage`: 토큰 사용량 정보
	- `response.id`: 응답의 고유 식별자
	- `response.model`: 사용된 모델 정보


In [4]:
# OpenAI API를 직접 사용하는 방법 (에러처리 포함)
from openai import OpenAI
from openai import OpenAIError, APIError, RateLimitError, APIConnectionError
import os

# 통신을 위한 클라이언트 생성 (.env 파일을 사용하지 않는 경우에는 주석을 해제하고 api_key를 직접 입력)
client = OpenAI(
	api_key= os.getenv("OPENAI_API_KEY")
)

try: 
	# Completions요청 (prompt > completion)
	response = client.chat.completions.create(
		model="gpt-4.1-mini-2025-04-14",
		messages=[
			# system 역할 : 전반적인 동작 방식 정의(GPT-4 계열에서 권장)
			{"role": "system", "content": "You are a helopful programming assiant"},
			# user 역할 : 실제 요청 내용
			{"role": "user", "content": "오늘 날짜는 언제인가요?"},
		],
		temperature=0.7,
		max_tokens=2000,
	)

	# 결과 출력
	print("response: ", response)
	print("id: ", response.id)
	print("model: ", response.model)
	print("text: ", response.choices[0].message.content)
	print("usage: ", response.usage)


except RateLimitError as e:
    print(f"⚠️ Rate Limit 오류: API 사용량 한도를 초과했습니다. 잠시 후 다시 시도해주세요.")
    print(f"상세 정보: {e}")
except APIConnectionError as e:
    print(f"⚠️ 연결 오류: OpenAI API 서버에 연결할 수 없습니다. 네트워크 연결을 확인해주세요.")
    print(f"상세 정보: {e}")
except APIError as e:
    print(f"⚠️ API 오류: OpenAI API에서 오류가 발생했습니다.")
    print(f"상세 정보: {e}")
except OpenAIError as e:
    print(f"⚠️ OpenAI 오류: {e}")
except Exception as e:
    print(f"⚠️ 예상치 못한 오류가 발생했습니다: {e}")

response:  ChatCompletion(id='chatcmpl-D6TQNF73dhymwlwrigCxEwAodo79C', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='오늘 날짜는 2024년 4월 27일입니다. 도움이 필요하시면 언제든지 말씀해 주세요!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1770435447, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_75546bd1a7', usage=CompletionUsage(completion_tokens=26, prompt_tokens=27, total_tokens=53, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
id:  chatcmpl-D6TQNF73dhymwlwrigCxEwAodo79C
model:  gpt-4.1-mini-2025-04-14
text:  오늘 날짜는 2024년 4월 27일입니다. 도움이 필요하시면 언제든지 말씀해 주세요!
usage:  CompletionUsage(completion_tokens=26, prompt_tokens=27, total_tokens=53, completion_tokens_detail

In [5]:
from IPython.display import Markdown, display

display(Markdown(response.choices[0].message.content))

오늘 날짜는 2024년 4월 27일입니다. 도움이 필요하시면 언제든지 말씀해 주세요!

#### 4.2 구조화된 JSON출력
구조화된 출력은 데이터 처리와 분석에 용이하며, API 응답의 일관성을 보장합니다.
- JSON Schema 정의
	- `response_format`을 통해 응답의 형식을 JSON으로 지정합니다.
	- `json_schema`에서 데이터 구조와 각 필드의 특성을 정의합니다.
		- type: 데이터 타입 - string, number
		- description: 각 필드에 대한 설명
		- required: 필수 필드 지정
		- additionalProperties: 추가 속성 허용 여부
- 정보 추출 과정
	- 입력된 텍스트에서 정규화된 형태로 정보를 추출
	- 지정된 스키마에 맞춰 JSON객체 구성
	- 필수 필드가 누락되지 않도록 검증
	- 가격과 같은 숫자 정보는 적절한 형식으로 변환

In [5]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4.1-mini-2025-04-14",
    messages=[
        {
            "role": "system",
            "content": "상품 정보를 구조화된 형태로 추출하고, 각 속성에 대해 자세히 설명합니다."
        },
        {
            "role": "user",
            "content": "애플 아이폰 15 프로 256GB (블랙) - 1,500,000원"
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "product_schema",
            "description": "상품의 상세 정보를 구조화하기 위한 스키마",
            "schema": {
                "type": "object",
                "properties": {
                    "brand": {
                        "type": "string",
                        "description": "제조사 또는 브랜드 이름 (예: 애플, 삼성, LG 등)"
                    },
                    "model": {
                        "type": "string",
                        "description": "제품의 모델명 또는 시리즈명"
                    },
                    "capacity": {
                        "type": "string",
                        "description": "저장 용량 또는 규격 (예: 256GB, 512GB 등)"
                    },
                    "color": {
                        "type": "string",
                        "description": "제품의 색상"
                    },
                    "price": {
                        "type": "number",
                        "description": "제품의 가격 (단위: 원)",
                        "minimum": 0
                    },
                    "category": {
                        "type": "string",
                        "description": "제품의 카테고리 (예: 스마트폰, 노트북 등)"
                    },
                },
                "required": ["brand", "model", "price"],
                "additionalProperties": False
            }
        }
    }
)

# 결과 출력
print("response: ", response)
print("response type : ", type(response))
print("id:", response.id)
print('model:', response.model)
print("text:", response.choices[0].message.content)
print("usage:", response.usage)

response:  ChatCompletion(id='chatcmpl-D6Tpyb3WLLLfveMUocdRfilhkgcAX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"brand":"애플","model":"아이폰 15 프로","capacity":"256GB","color":"블랙","price":1500000,"category":"스마트폰"}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1770437034, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_75546bd1a7', usage=CompletionUsage(completion_tokens=36, prompt_tokens=404, total_tokens=440, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
response type :  <class 'openai.types.chat.chat_completion.ChatCompletion'>
id: chatcmpl-D6Tpyb3WLLLfveMUocdRfilhkgcAX
model: gpt-4.1-mini-2025-04-14
text: {"brand":"애플","model":"아이

In [4]:
# 응답을 json형태로 출력하기
import json


data = json.loads(response.choices[0].message.content)
data

{'brand': '애플',
 'model': '아이폰 15 프로',
 'capacity': '256GB',
 'color': '블랙',
 'price': 1500000,
 'category': '스마트폰'}